# 10 notebook that applies all project cards (replacing notebooks 02-09)

In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy
import re

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc
from lasso import util

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
console_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(console_handler)
logger.setLevel(logging.INFO)

# Remote I/O and parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_12')
output_dir = os.path.join(root_dir, 'data', 'processed', 'version_12_with_link21_fixes')
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [6]:
parameters.zones

4756

In [7]:
parameters.output_proj

<Projected CRS: EPSG:2875>
Name: NAD83(HARN) / California zone 6 (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - California - counties Imperial; Orange; Riverside; San Diego.
- bounds: (-118.15, 32.53, -114.42, 34.08)
Coordinate Operation:
- name: SPCS83 California zone 6 (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: NAD83 (High Accuracy Reference Network)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

# Read pickle

In [8]:
version_01_pickle_file_name = os.path.join(input_dir, 'working_scenario_01.pickle')
v_01_scenario = pickle.load(open(version_01_pickle_file_name, 'rb'))

In [9]:
v_01_scenario.transit_net.feed.stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21881 entries, 0 to 21880
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   stop_name            21881 non-null  object 
 1   stop_lat             21881 non-null  float64
 2   stop_lon             21881 non-null  float64
 3   zone_id              8233 non-null   object 
 4   agency_raw_name      21881 non-null  object 
 5   stop_code            15289 non-null  object 
 6   location_type        5475 non-null   float64
 7   parent_station       249 non-null    object 
 8   stop_desc            409 non-null    object 
 9   stop_url             261 non-null    object 
 10  stop_timezone        126 non-null    object 
 11  wheelchair_boarding  377 non-null    float64
 12  platform_code        52 non-null     object 
 13  position             0 non-null      object 
 14  direction            0 non-null      object 
 15  * used by routes     124 non-null   

In [10]:
v_01_scenario.transit_net.feed.stops[
    v_01_scenario.transit_net.feed.stops.stop_name == "San Jose"]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
5466,San Jose,37.329935,-121.902496,56146,Capitol_2017_3_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6680,NaN,NaN,2192825


# Create scenario and apply projects

waiting on dependencies to work

In [11]:
v_01_scenario.road_net.links_df["id"] = v_01_scenario.road_net.links_df["shstGeometryId"]

In [12]:
step1_scenario = Scenario.create_scenario(
    base_scenario=v_01_scenario,
    card_directory = card_dir,
    tags = ['highway review', "Major Arterial Review", "Reversible Lanes", 'Bus Only', 'Toll Plaza'],
    validate_project_cards=False)

Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [13]:
step1_scenario.apply_all_projects()

Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_service_road_dumbarton_bridge_bypass.yml]:Year 2015 Modify Service Road Dumbarton Bridge Bypass
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_service_road_dumbarton_bridge_bypass.yml]:Year 2015 Modify Service Road Dumbarton Bridge Bypass
Applying Project to Roadway Network: Year 2015 Modify Service Road Dumbarton Bridge Bypass
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_service_road_carquinez_bridge_bypass.yml]:Year 2015 Modify Service Road Carquinez Bridge Bypass
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_service_road_carquinez_bridge_bypass.yml]:Year 2015 Modify Service Road Carquinez Bridge Bypass
Applying Project to Roadway Network: Year 2015 Modify Service Road Carquinez Bridge Bypass
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_san_mateo_bridge_ad

c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Applying Project to Transit Network: add san mateo bridge toll plaza
Roadway Deletion not yet implemented in Transit; ignoring
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_san_mateo_bridge_add_plaza_link.yml]:add san mateo bridge toll plaza
Applying Project to Roadway Network: add san mateo bridge toll plaza
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_lanes_ft_correct_as_is.yml]:Year 2015 Correct As Is
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_lanes_ft_correct_as_is.yml]:Year 2015 Correct As Is
Applying Project to Roadway Network: Year 2015 Correct As Is
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_change_should_be_service_lanes_correct.yml]:Year 2015 Modify Facility Type to Service
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_c

c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351:

Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_change_should_be_ramp_lanes_correct.yml]:Year 2015 Modify Facility Type to Ramp
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_change_should_be_ramp_lanes_correct.yml]:Year 2015 Modify Facility Type to Ramp
Applying Project to Roadway Network: Year 2015 Modify Facility Type to Ramp
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_change_should_be_ramp_2_lanes_shape_incorrect.yml]:Year 2015 Modify Lanes to 2 Lane and Facility Type to Ramp shape incorrect
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_change_should_be_ramp_2_lanes_shape_incorrect.yml]:Year 2015 Modify Lanes to 2 Lane and Facility Type to Ramp shape incorrect
Applying Project to Roadway Network: Year 2015 Modify Lanes to 2 Lane and Facility Type to Ramp shape incorrect


...

Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_golden_gate_bridge_plaza_add.yml]:add golden gate bridge toll plaza
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_golden_gate_bridge_plaza_add.yml]:add golden gate bridge toll plaza
Applying Project to Roadway Network: add golden gate bridge toll plaza
Applying Project to Transit Network: add golden gate bridge toll plaza
Roadway Deletion not yet implemented in Transit; ignoring
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_golden_gate_bridge_plaza_add.yml]:add golden gate bridge toll plaza
Applying Project to Roadway Network: add golden gate bridge toll plaza
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_bus_only_transbay_terminal_ramps.yml]:Year 2015 Bus Only Segments for Transbay Terminal Ramps
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_bus_only_transba

c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\sterling_on_ramp_hov_only.wrangler]:year 2015 sterling on ramp HOV only in PM
Applying Project to Roadway Network: year 2015 sterling on ramp HOV only in PM
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\reversible_lanes_golden_gate_bridge.wrangler]:year 2015 golden gate reversible lanes
Applying Project to Roadway Network: year 2015 golden gate reversible lanes


c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\golden_gate_bridge_shape.wrangler]:year 2015 golden gate bridge shape correction
Applying Project to Roadway Network: year 2015 golden gate bridge shape correction


In [14]:
step1_scenario.road_net.links_df[step1_scenario.road_net.links_df.model_link_id == 8063198][["A", "B", "geometry", "id"]]

,A,B,geometry,id
1634794,5013515,5030001,"LINESTRING (-122.48221 37.83307, -122.48175 37...",d01b27cabff869e0bf4f7187ee77d6de


In [15]:
step1_scenario.road_net.nodes_df[step1_scenario.road_net.nodes_df.model_node_id == 5030001]

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry,farezone
644481,NaN,,Marin,1,0,0,5030001,0,-122.47658,37.808073,POINT (-122.47658 37.80807),


In [16]:
list(step1_scenario.road_net.links_df[step1_scenario.road_net.links_df.model_link_id == 8063198].geometry.iloc[0].coords)

[(-122.4822082, 37.8330742),
 (-122.48175233602524, 37.832883609672905),
 (-122.48116359114647, 37.83254572138074),
 (-122.4807545542717, 37.8322078315411),
 (-122.48029455542564, 37.83168881278591),
 (-122.48009875416754, 37.83132443825398),
 (-122.47998476028442, 37.83103844535508),
 (-122.47989490628242, 37.83066029748685),
 (-122.47976046055554, 37.829561062531596),
 (-122.47972592711449, 37.82925917389109),
 (-122.47969474643469, 37.82899541754155),
 (-122.47965652495623, 37.82866942822046),
 (-122.47961830347776, 37.828343967096075),
 (-122.4795787408948, 37.82800393945721),
 (-122.47953817248343, 37.82765676008782),
 (-122.47949860990049, 37.827314081061175),
 (-122.47865572571753, 37.820096795765785),
 (-122.4780549108982, 37.81498611149333),
 (-122.47746214270592, 37.80990050232683),
 (-122.47741788625717, 37.80948516210632),
 (-122.47734010219573, 37.80920968005912),
 (-122.47717916965483, 37.80883671811151),
 (-122.47690826654433, 37.808453158689176),
 (-122.47657969594002, 

In [17]:
list(step1_scenario.road_net.shapes_df[step1_scenario.road_net.shapes_df.id == "d01b27cabff869e0bf4f7187ee77d6de"].geometry.iloc[0].coords)

[(-122.4822082, 37.8330742),
 (-122.48175233602524, 37.832883609672905),
 (-122.48116359114647, 37.83254572138074),
 (-122.4807545542717, 37.8322078315411),
 (-122.48029455542564, 37.83168881278591),
 (-122.48009875416754, 37.83132443825398),
 (-122.47998476028442, 37.83103844535508),
 (-122.47989490628242, 37.83066029748685),
 (-122.47976046055554, 37.829561062531596),
 (-122.47972592711449, 37.82925917389109),
 (-122.47969474643469, 37.82899541754155),
 (-122.47965652495623, 37.82866942822046),
 (-122.47961830347776, 37.828343967096075),
 (-122.4795787408948, 37.82800393945721),
 (-122.47953817248343, 37.82765676008782),
 (-122.47949860990049, 37.827314081061175),
 (-122.47865572571753, 37.820096795765785),
 (-122.4780549108982, 37.81498611149333),
 (-122.47746214270592, 37.80990050232683),
 (-122.47741788625717, 37.80948516210632),
 (-122.47734010219573, 37.80920968005912),
 (-122.47717916965483, 37.80883671811151),
 (-122.47690826654433, 37.808453158689176),
 (-122.47657969594002, 

In [18]:
step2_scenario = Scenario.create_scenario(
    base_scenario = step1_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ['Managed Lanes', 'toll review', 'Exclude Trucks'],
    validate_project_cards=False)

Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [19]:
step2_scenario.apply_all_projects()

Projects year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
Projects year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass has add san mateo bridge toll plaza as prerequisite project which is missing for the scenario
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_san_mateo_bridge_toll_plaza.yml]:year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_san_mateo_bridge_toll_plaza.yml]:year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass
Applying Project to Roadway Network: year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass
Applying [D:/github/lfs-

c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_richmond_bridge_toll_plaza.yml]:Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_richmond_bridge_toll_plaza.yml]:Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_us101s_segment02_south_healdsburg_to_north_petaluma.yml]:Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_us101s_segment02_south_healdsburg_to_north_petaluma.yml]:Year 2015 Modify USE and HOV Lanes on US-101 SB (S

c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351:

Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_us101s_novata_to_north_marin_city.yml]:Year 2015 Modify HOV Lanes and USE on US-101 SB from Novato to North of Marin City
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_us101s_novata_to_north_marin_city.yml]:Year 2015 Modify HOV Lanes and USE on US-101 SB from Novato to North of Marin City
Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on US-101 SB from Novato to North of Marin City


...

c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_carniquez_bridge_toll_plaza.yml]:Year 2015 Modify HOV Lanes and USE on Carquinez Bridge Toll Plaza on I-80 NB - Bridge Pass
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_carniquez_bridge_toll_plaza.yml]:Year 2015 Modify HOV Lanes and USE on Carquinez Bridge Toll Plaza on I-80 NB - Bridge Pass
Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Carquinez Bridge Toll Plaza on I-80 NB - Bridge Pass
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_attributes_exclude_trucks_sr85.yml]:Year 2015 Exclude Trucks on SR 85
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_attributes_exclude_trucks_sr85.yml]:Year 2015 Exclude Trucks on SR 85
Applying Project to Roadway Network: Year 2015 Exclude Trucks on SR 85


c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_attributes_exclude_trucks_i580.yml]:Year 2015 Exclude Trucks on I580
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_attributes_exclude_trucks_i580.yml]:Year 2015 Exclude Trucks on I580
Applying Project to Roadway Network: Year 2015 Exclude Trucks on I580
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_antioch_bridge.yml]:Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on Antioch Bridge NB on Route 160 - Bridge Pass
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_antioch_bridge.yml]:Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on Antioch Bridge NB on Route 160 - Bridge Pass
Applying Project to Roadway Network: Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on Antioch Bridge NB on Route 160 - Bridge Pass


c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [20]:
step2_scenario.road_net.links_df.shape

(1634796, 44)

In [21]:
step2_scenario.road_net.links_df.model_link_id.nunique()

1634796

In [22]:
step2_scenario.road_net.links_df[step2_scenario.road_net.links_df.model_link_id == 3149471].useclass

762806    {'default': 0, 'timeofday': [{'time': (21600, ...
Name: useclass, dtype: object

In [23]:
step2_scenario.road_net.links_df.county.isnull().sum()

0

In [24]:
step2_scenario.road_net.links_df.model_link_id.max()

8063199

In [25]:
step3_scenario = Scenario.create_scenario(
    base_scenario= step2_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ["Major Transit links"],
    validate_project_cards=False)

step3_scenario.apply_all_projects()

Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\rail-to-street-connector-links.yml]:move street rail to street connector links
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\rail-to-street-connector-links.yml]:move street rail to street connector links
Applying Project to Roadway Network: move street rail to street connector links
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\rail-to-street-connector-links.yml]:move street rail to street connector links
Applying Project to Roadway Network: move street rail to street connector links


c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\rail-to-street-connector-links.yml]:move street rail to street connector links
Applying Project to Roadway Network: move street rail to street connector links
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\rail-to-street-connector-links.yml]:move street rail to street connector links
Applying Project to Roadway Network: move street rail to street connector links
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\rail-to-street-connector-links.yml]:move street rail to street connector links
Applying Project to Roadway Network: move street rail to street connector links
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\rail-to-street-connector-links.yml]:move street rail to street connector links
Applying Project to Roadway Network: move street rail to street connector links
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards

In [26]:
step3_scenario.road_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'geometry', 'ft_cal', 'ft', 'useclass',
       'assignable_cal', 'assignable', 'transit', 'id', 'tollbooth',
       'bus_only', 'transit_access', 'managed', 'ML_lanes', 'segment_id',
       'ML_tollbooth', 'ML_useclass', 'ML_access', 'ML_egress', 'ML_tollseg',
       'tollseg'],
      dtype='object')

In [27]:
step3_scenario.road_net.links_df.shape

(1634814, 44)

In [28]:
step3_scenario.road_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'geometry', 'ft_cal', 'ft', 'useclass',
       'assignable_cal', 'assignable', 'transit', 'id', 'tollbooth',
       'bus_only', 'transit_access', 'managed', 'ML_lanes', 'segment_id',
       'ML_tollbooth', 'ML_useclass', 'ML_access', 'ML_egress', 'ML_tollseg',
       'tollseg'],
      dtype='object')

In [29]:
step3_scenario.transit_net.set_roadnet(step3_scenario.road_net, 
                                      validate_consistency = False)

step4_scenario = Scenario.create_scenario(
    base_scenario= step3_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ["Minor Transit", "Add Transit", "Major Transit", "Toll Plaza Transit"],
    validate_project_cards=False)

step4_scenario.apply_all_projects()

Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


Projects sfmta route n-judah has move street rail to street connector links as prerequisite project which is missing for the scenario
Projects update routing for golden gate bridge toll plaza transit has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
Projects update routing for san mateo bridge toll plaza transit has add san mateo bridge toll plaza as prerequisite project which is missing for the scenario
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_san_mateo_bridge_toll_plaza_transit.yml]:update routing for san mateo bridge toll plaza transit
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_san_mateo_bridge_toll_plaza_transit.yml]:update routing for san mateo bridge toll plaza transit
Applying Project to Transit Network: update routing for san mateo bridge toll plaza transit
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_golden_gate_b

c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


Creating a new stop in stops.txt for node ID: 2104183
Creating a new stop in stops.txt for node ID: 2015328
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\VTA_22.yml]:VTA Route 22
Applying Project to Transit Network: VTA Route 22
Missing connections from node 2033565 to node 2103889 for the new routing, find complete path using default graph
Routing path from node 2033565 to node 2103889 for missing connections: ['2033565', '2155466', '2103889'].
Missing connections from node 2035353 to node 2158621 for the new routing, find complete path using default graph
Routing path from node 2035353 to node 2158621 for missing connections: ['2035353', '2061243', '2158621'].
Missing connections from node 2026323 to node 2102903 for the new routing, find complete path using default graph
Routing path from node 2026323 to node 2102903 for missing connections: ['2026323', '2045820', '2102903'].
Creating a new stop in stops.txt for node ID: 2103889
Creating a new stop in stop

...

Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\ac-transit-route-40-edits.yml]:year 2015 ac transit route 40 edits
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\ac-transit-route-40-edits.yml]:year 2015 ac transit route 40 edits
Applying Project to Transit Network: year 2015 ac transit route 40 edits
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\ac-transit-route-38-edits.yml]:year 2015 ac transit route 38 edits
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\ac-transit-route-38-edits.yml]:year 2015 ac transit route 38 edits
Applying Project to Transit Network: year 2015 ac transit route 38 edits
Creating a new stop in stops.txt for node ID: 2503531
Creating a new stop in stops.txt for node ID: 2539453
Creating a new stop in stops.txt for node ID: 2564896


c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\ac-transit-route-06-edits.yml]:year 2015 ac transit route 06 edits
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\ac-transit-route-06-edits.yml]:year 2015 ac transit route 06 edits
Applying Project to Transit Network: year 2015 ac transit route 06 edits
Creating a new stop in stops.txt for node ID: 2592196
Creating a new stop in stops.txt for node ID: 2606436
Creating a new stop in stops.txt for node ID: 2519741
Creating a new stop in stops.txt for node ID: 2571302
Creating a new stop in stops.txt for node ID: 2576003
Creating a new stop in stops.txt for node ID: 2521855


c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\ac-transit-fare-system-change.wrangler]:ac transit fare system change to zonal fare
Applying Project to Transit Network: ac transit fare system change to zonal fare


In [32]:
step4_scenario.transit_net.feed.stop_times[step4_scenario.transit_net.feed.stop_times.trip_id == "5877"]
step4_scenario.transit_net.feed.frequencies.loc[step4_scenario.transit_net.feed.frequencies.trip_id == "116", "headway_secs"]
step4_scenario.transit_net.feed.stops.loc[(step4_scenario.transit_net.feed.stops.stop_name == "19th St. Oakland") & (step4_scenario.transit_net.feed.stops.agency_raw_name == "BART_2015_8_3"), "model_node_id"]

5354    2625944
5355    2625944
Name: model_node_id, dtype: object

In [33]:
step4_scenario.road_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = step4_scenario.road_net,
    parameters = parameters)

Adding centroid and centroid connector to standard network
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Finished adding centroid and centroid connectors


In [34]:
print(step4_scenario.road_net.links_df.shape)
print(step4_scenario.road_net.links_df.model_link_id.nunique())
print(step4_scenario.road_net.nodes_df.shape)
print(step4_scenario.road_net.nodes_df.model_node_id.nunique())
print(step4_scenario.road_net.shapes_df.shape)

(2199316, 44)
2199316
(688964, 12)
688964
(1152265, 6)


In [35]:
step4_scenario.road_net.links_df.locationReferences.iloc[-1]

[{'sequence': 1,
  'point': [-122.50580293859502, 37.94830473568058],
  'distanceToNextRef': 51.382886808196375,
  'bearing': 0,
  'intersectionId': nan},
 {'sequence': 2,
  'point': [-122.50576480000001, 37.9487303],
  'intersectionId': '44c53879b5a9797cbb2eff72b5f64699'}]

In [36]:
step5_scenario = Scenario.create_scenario(
    base_scenario= step4_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ['External Stations Review'],
    validate_project_cards=False)

step5_scenario.apply_all_projects()

Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_william_elton_brown_freeway_I-580_centroid_connector SB.yaml]:Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_william_elton_brown_freeway_I-580_centroid_connector SB.yaml]:Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_william_elton_brown_freeway_I-580_centroid_connector NB.yaml]:Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
Applying [D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_r

In [37]:
print(step5_scenario.road_net.links_df.shape)
print(step5_scenario.road_net.links_df.model_link_id.nunique())
print(step5_scenario.road_net.nodes_df.shape)
print(step5_scenario.road_net.nodes_df.model_node_id.nunique())

(2199342, 44)
2199342
(688964, 12)
688964


In [39]:
# add taps and tap connectors
step5_scenario.road_net = mtc.add_tap_and_tap_connector(
    roadway_network = step5_scenario.road_net,
    transit_network = step5_scenario.transit_net, 
    parameters = parameters)

Adding tap and tap connector to standard network
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
There are 601 stops not connected to taps, connecting them to the closest taps.
snapping gtfs stops to roadway node osmid...


z:\data\users\sijia\mtc\github\lasso\lasso\mtc.py:2812: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  )


Finished adding tap and tap connectors


In [40]:
step5_scenario.road_net.links_df[step5_scenario.road_net.links_df.B == 1027676][['A', 'B']]

,A,B
1632933,1027675,1027676
1633620,1007182,1027676
2221259,90035,1027676


In [41]:
step5_scenario.road_net.nodes_df[step5_scenario.road_net.nodes_df.model_node_id == 1027677]

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry,farezone
643982,NaN,nan,San Francisco,0.0,1.0,0.0,1027677,1.0,-122.415307,37.805257,POINT (-122.41531 37.80526),NaN


In [42]:
# add county

step5_scenario.road_net = mtc.calculate_county(
    roadway_network = step5_scenario.road_net,
    parameters = parameters,
    network_variable = 'county'
)

County Variable 'county' already in network, calculating for missing ones
Adding roadway network variable for county using a spatial join with: Z:/Data/Users/Sijia/MTC/github/Lasso\mtc_data\county\county.shp


z:\data\users\sijia\mtc\github\lasso\lasso\mtc.py:2979: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  link_centroids_gdf["geometry"] = link_centroids_gdf["geometry"].centroid
c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Finished Calculating link county variable: county


c:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Finished Calculating node county variable: county


In [43]:
step5_scenario.road_net.links_df.county.value_counts()

Santa Clara      660008
Alameda          443532
Contra Costa     326621
San Mateo        203882
Sonoma           178792
Solano           157454
San Francisco    133492
Marin             84883
Napa              47068
External             48
Name: county, dtype: int64

In [45]:
step5_scenario.road_net.nodes_df.county.value_counts()

Santa Clara      203748
Alameda          137128
Contra Costa     104665
San Mateo         61887
Sonoma            59970
Solano            50710
San Francisco     33406
Marin             28439
Napa              14990
External             21
Name: county, dtype: int64

# sanity check

In [47]:
step5_scenario.road_net.links_df[step5_scenario.road_net.links_df.model_link_id == 3150085] # should be ramp lanes 2

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
765902,,False,True,d1b76ff92ba7a2915de8efc97c782d73,2,,,True,,motorway_link,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
step5_scenario.road_net.links_df[step5_scenario.road_net.links_df.model_link_id == 3107408] # bay bridge toll link

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
548523,"['nan', 'nan']",False,True,826a99158fd58e5483d1085c9e0d65b3,"{'default': 4, 'timeofday': [{'time': (21600, ...","['nan', 'nan']","['nan', 'nan']","[True, True]","['I 80', 'I 80']",motorway,...,NaN,1,"{'default': 0, 'timeofday': [{'time': (21600, ...",5,5,"{'default': 0, 'timeofday': [{'time': (21600, ...",all,all,NaN,NaN


In [49]:
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 26067343
].iloc[0][["A", "B"]]

A     900020
B    2192724
Name: 2199322, dtype: object

In [50]:
# this link is deleted and splitted into two for toll plaza at GGB

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8016300]

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg


In [51]:
# golden gate bridge

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8063198].lanes.iloc[0]

{'default': 3,
 'timeofday': [{'time': (21600, 36000), 'value': 4},
  {'time': (54000, 68400), 'value': 2}]}

In [52]:
# golden gate bridge HOV

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8063199].ML_lanes.iloc[0]

{'default': 0,
 'timeofday': [{'time': (21600, 36000), 'value': 2},
  {'time': (54000, 68400), 'value': 2}]}

In [53]:
# check if new node is created for toll plaza at GGB

step5_scenario.road_net.nodes_df[step5_scenario.road_net.nodes_df.model_node_id == 5030001]

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry,farezone
644481,NaN,,Marin,1.0,0.0,0.0,5030001,0.0,-122.47658,37.808073,POINT (-122.47658 37.80807),


In [54]:
# golden gate bridge toll plaza

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8063199].lanes.iloc[0]

{'default': 9,
 'timeofday': [{'time': (21600, 36000), 'value': 7},
  {'time': (54000, 68400), 'value': 7}]}

In [55]:
# check shapes for new links
# example: toll plaza at GGB and a new rail-to-street link

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id.isin([8063198, 8063199, 26067378])][["id", "geometry"]]

,id,geometry
1634794,d01b27cabff869e0bf4f7187ee77d6de,"LINESTRING (-122.48221 37.83307, -122.48175 37..."
1634795,b46d3b9a4683bfe26b97e2a66181d8dd,"LINESTRING (-122.47658 37.80807, -122.47536 37..."
1634796,ba0e166e3a9f6c1cc8d7e331415769b2,"LINESTRING (-122.42941 37.76947, -122.43026 37..."


In [56]:
step5_scenario.road_net.shapes_df[
    step5_scenario.road_net.shapes_df.id.isin(["b46d3b9a4683bfe26b97e2a66181d8dd", "ba0e166e3a9f6c1cc8d7e331415769b2"])][
        ["id", "geometry"]]

,id,geometry
869991,b46d3b9a4683bfe26b97e2a66181d8dd,"LINESTRING (-122.47658 37.80807, -122.47536 37..."
869992,ba0e166e3a9f6c1cc8d7e331415769b2,"LINESTRING (-122.42941 37.76947, -122.43026 37..."


In [57]:
step5_scenario.transit_net.feed.shapes.shape_id.nunique()

1592

In [58]:
step5_scenario.transit_net.feed.shapes[
    step5_scenario.transit_net.feed.shapes.shape_id.isin(
        ["1169", "1168", "939", "940", "943", "944"]
    )
].groupby(["shape_id"]).count()

,shape_pt_sequence,shape_osm_node_id,shape_shst_node_id,shape_model_node_id,shape_pt_lat,shape_pt_lon
shape_id,,,,,,
1168,177,108,108,177,0,0
1169,176,65,65,176,0,0
939,26,22,22,26,0,0
940,161,157,157,161,0,0
943,158,154,154,158,0,0
944,130,128,128,130,0,0


In [59]:
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.A.isin([2579949, 2607569])][["A", "B", "roadway","assignable"]]

,A,B,roadway,assignable
514595,2579949,2535246,residential,0.0
842601,2579949,2604856,residential,1.0
846406,2607569,2586861,residential,1.0
1009927,2607569,2546185,residential,1.0
1077889,2607569,2541364,residential,0.0
1645033,2579949,300963,taz,1
1645951,2607569,300963,taz,1
1820125,2579949,318103,maz,1
1820126,2579949,327248,maz,1
1820127,2579949,328044,maz,1


In [60]:
# check Service Road Dumbarton Bridge Bypass is not assignable
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 3152465][["A", "B", "roadway","assignable"]]

,A,B,roadway,assignable
778002,2552715,2540313,motorway_link,0.0


In [61]:
# check should be 3 lanes

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 2217343][["A", "B", "roadway","assignable", "lanes"]]

,A,B,roadway,assignable,lanes
690625,2153360,2071468,motorway,1.0,"{'default': 3, 'timeofday': [{'time': (21600, ..."


In [62]:
# should be ramps
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id.isin(
        [3031886,3006561,3046803,3101610,3011518,3024387,4018032])][["A", "B", "roadway","assignable", "ft", "walk_access"]]

,A,B,roadway,assignable,ft,walk_access
33757,2506600,2542798,motorway_link,1.0,3,1
59112,2511462,2523557,motorway_link,1.0,3,1
123885,3017507,3058438,motorway_link,1.0,3,1
124668,2523557,2549665,motorway_link,1.0,3,1
162620,2501507,2506600,motorway_link,1.0,3,1
238525,2542798,2518014,motorway_link,1.0,3,1
518512,2580420,2511462,motorway_link,1.0,3,1


In [63]:
# zone id should be "2" (ac transit), "1" (dumbarton union city), "2" (ac transit dumbarton), "1" (samtrans) , "1" (samtrans)

step5_scenario.transit_net.feed.stops[step5_scenario.transit_net.feed.stops.stop_id.isin(
    ["101523872", "4694", "4669", "102036044", "4187"])]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
4130,Transbay Temp Terminal,37.789707,-122.393626,2,ACTransit_2015_8_14,50751,NaN,place_ETSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4187,3955000020,459ceb872e72e49fd4041a78884935e2,1021597
4611,Union City BART,37.590360,-122.017465,1,ACTransit_2015_8_14,50781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4669,1025879893,84c3f377f9f0065bf14be3af92202d0d,2509987
4636,Stanford Oval,37.429272,-122.168925,2,ACTransit_2015_8_14,55432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4694,522171022,6d2aeffcbad07f0873a268bea42838f3,2149155
22309,NaN,37.444794,-122.168849,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102036044,NaN,NaN,2036044
22338,NaN,37.673730,-122.467029,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101523872,NaN,NaN,1523872


In [64]:
# check zone_id is removed for gg ferry stops

step5_scenario.transit_net.feed.stops[step5_scenario.transit_net.feed.stops.stop_id.isin(
    ["7706", "7707", "7708"])]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
7603,San Franciso Ferry Terminal,37.796169,-122.391617,NaN,GGTransit_2015_9_3,43000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7706,NaN,NaN,1027623
7604,Sausalito Ferry Terminal,37.856403,-122.478249,NaN,GGTransit_2015_9_3,43003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7707,NaN,NaN,5026533
7605,Larkspur Ferry Terminal,37.945316,-122.508736,NaN,GGTransit_2015_9_3,43004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7708,NaN,NaN,5026532


In [65]:
# check zone_id is updated for fairfield

step5_scenario.transit_net.feed.stops[step5_scenario.transit_net.feed.stops.stop_id.isin(
    ["6817", "6972"])]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
6741,El Cerrito Del Norte BART,37.924894,-122.317048,671,Fairfield_2015_10_14,400,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6972,1781110705,e235e2f09ba74bdbe6b6abb3e44fc47e,3060051
6779,Main Street and Lotz Way,38.243081,-122.040478,663,Fairfield_2015_10_14,172,0.0,NaN,Far-Side,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6817,62481309,52b70c286cb2792528a436f5736b33f7,3512083


In [66]:
# check USECLASS on I-80 WB to Bay Bridge Toll Plaza

step5_scenario.road_net.links_df[step5_scenario.road_net.links_df['model_link_id'].isin([3061568, 3149471])].useclass

314471    {'default': 0, 'timeofday': [{'time': (21600, ...
762806    {'default': 0, 'timeofday': [{'time': (21600, ...
Name: useclass, dtype: object

In [67]:
# check zone-id for new stops on SamTrans
step5_scenario.transit_net.feed.stops[
    step5_scenario.transit_net.feed.stops.stop_id.astype(int).isin(
    [102036044,101511493,101516920,101525498,101516256,101513094,101513976,101528479,101529690,101529454,101519622,101513122,101514054,101543182,101512385,101534620,101503246,101537667,101538028,101526122,101519051,101503077,101508313,101508126,101537888,101519350,101516319,101509775,101536669])
]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
22309,NaN,37.444794,-122.168849,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102036044,NaN,NaN,2036044
22310,NaN,37.472389,-122.216100,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101511493,NaN,NaN,1511493
22311,NaN,37.483308,-122.231206,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101516920,NaN,NaN,1516920
22312,NaN,37.485404,-122.233938,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101525498,NaN,NaN,1525498
22313,NaN,37.489469,-122.238568,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101516256,NaN,NaN,1516256
22314,NaN,37.503227,-122.255333,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101513094,NaN,NaN,1513094
22315,NaN,37.507359,-122.260283,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101513976,NaN,NaN,1513976
22316,NaN,37.526752,-122.283581,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101528479,NaN,NaN,1528479
22317,NaN,37.540371,-122.301416,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101529690,NaN,NaN,1529690
22318,NaN,37.559804,-122.322379,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101529454,NaN,NaN,1529454


In [68]:
# check zone-id for new stops on SamTrans
step5_scenario.transit_net.feed.stops[
    step5_scenario.transit_net.feed.stops.stop_id.astype(int).isin(
    [101523872,101545076,101503345,101519909,101504028,101514714,101506746])
]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
22338,NaN,37.673730,-122.467029,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101523872,NaN,NaN,1523872
22339,NaN,37.682409,-122.487089,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101545076,NaN,NaN,1545076
22340,NaN,37.697643,-122.483092,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101503345,NaN,NaN,1503345
22341,NaN,37.701361,-122.484757,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101519909,NaN,NaN,1519909
22342,NaN,37.702417,-122.481156,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101504028,NaN,NaN,1504028
22343,NaN,37.704069,-122.475537,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101514714,NaN,NaN,1514714
22344,NaN,37.671171,-122.464817,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101506746,NaN,NaN,1506746


In [69]:
# check San Mateo bridge toll plaza
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id.isin(
    [3315463,3315462, 3024600])
].lanes.tolist()

[{'default': 4,
  'timeofday': [{'time': (21600, 36000), 'value': 3},
   {'time': (54000, 68400), 'value': 3}]},
 {'default': 4,
  'timeofday': [{'time': (21600, 36000), 'value': 3},
   {'time': (54000, 68400), 'value': 3}]},
 {'default': 7,
  'timeofday': [{'time': (21600, 36000), 'value': 5},
   {'time': (54000, 68400), 'value': 5}]}]

In [70]:
# check Bay Bridge west pan
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id.isin(
    [38545])
]

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
818708,"['nan', '']",False,True,dc196b0fb1ac8053a741cd53df6ff64d,5,"['50 mph', '']","['San Francisco – Oakland Bay Bridge', '']","[True, True]","['I 80', '']",motorway,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
step5_scenario.road_net.links_df.transit.value_counts()

0    1634035
1        753
          60
Name: transit, dtype: int64

In [72]:
step5_scenario.road_net.links_df.transit.value_counts()

0    1634035
1        753
          60
Name: transit, dtype: int64

In [74]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_12.pickle')
pickle.dump(step5_scenario, open(working_scenario_filename, 'wb'))

In [75]:
step5_scenario.road_net.links_df[step5_scenario.road_net.links_df.model_link_id == 3001245]

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
6341,"['nan', 'nan', 'nan']",False,True,afe9e007f73e3bf8740d67c62257c222,"{'default': 4, 'timeofday': [{'time': (21600, ...","['65 mph', '65 mph', '65 mph']","['nan', 'nan', 'nan']","[True, True, True]","['nan', 'I 680', 'I 680']",motorway,...,NaN,1,"{'default': 0, 'timeofday': [{'time': (21600, ...",25,25,"{'default': 0, 'timeofday': [{'time': (21600, ...","[2501277, 2585590]",[2585590],1,NaN


## Make Travel Model Networks

In [8]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_12.pickle')
step5_scenario = pickle.load(open(working_scenario_filename, 'rb'))

In [9]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = step5_scenario.road_net, 
    parameters = parameters)

Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Filling nan for network from network wrangler
Converting variable type to mtc standard


In [11]:
# Vallejo Ferry to TAP

model_net.links_df[model_net.links_df.B == 3541176][["A", "roadway", "county", "rail_only", "model_link_id"]]

,A,roadway,county,rail_only,model_link_id
191843,3512789,footway,Solano,0,5013621
1634815,1028039,,Contra Costa,1,26067374
1634817,1028040,,Contra Costa,1,26067376
2235714,590188,tap,Solano,0,5157453


In [12]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.nodes_df.model_node_id.nunique())
print(model_net.shapes_df.shape)
print(model_net.shapes_df.id.nunique())

(2235780, 44)
2235780
(694964, 12)
694964
(1170510, 6)
1170510


In [13]:
# update farezone due to AC Transit, Fairfield, gg ferries Fare change

model_net = mtc.calculate_farezone(
    roadway_network = model_net,
    transit_network = step5_scenario.transit_net,
    parameters = parameters,
    network_variable = 'farezone',
    overwrite = True,
)

model_net.nodes_df.farezone.value_counts()

Determining farezone
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Overwriting existing Variable 'farezone' already in network
Calculating and adding roadway network variable: farezone
Finished determining variable: farezone


7.0      4211
92.0     1551
85.0      225
74.0      208
105.0     179
         ... 
10.0        1
17.0        1
42.0        1
34.0        1
11.0        1
Name: farezone, Length: 124, dtype: int64

In [14]:
model_net.nodes_df[model_net.nodes_df.model_node_id.isin([1523872,1545076,1503345,1519909,1504028,1514714,1506746])].farezone

38492     92.0
46931     92.0
77849     92.0
169823    92.0
229436    92.0
275080    92.0
512571    92.0
Name: farezone, dtype: float64

In [15]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

Renaming roadway attributes to be consistent with what mtc's model is expecting
Creating managed lane network.
Creating network with duplicated managed lanes


C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\geopandas\geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


Determining cntype
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Calculating and adding roadway network variable: cntype
Finished determining variable: cntype
Determining transit
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Variable 'transit' updated for some links. Returning without overwriting for those links. Calculating for other links
Calculating and adding roadway network variable: transit
Finished determining transit-only variable: transit
Determining useclass
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Variable 'useclass' updated for some links. Returning without overwriting for those links. Calculating for other links
Calculating and adding roadway network variable: useclass
Finished determining variable: useclass
Calcul

z:\data\users\sijia\mtc\github\lasso\lasso\mtc.py:2979: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  link_centroids_gdf["geometry"] = link_centroids_gdf["geometry"].centroid
C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  """Entry point for launching an IPython kernel.


Finished Calculating link county variable: county


C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\lasso_bayarea\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  """Entry point for launching an IPython kernel.


Finished Calculating node county variable: county
Overwriting existing distance Variable 'distance' already in network
Calculating distance for all links
Calculating distance for all links
Filling nan for network from network wrangler
Splitting variables by time period and category
Specified variable to split: ML_lanes not in network variables: Index(['index', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'geometry', 'ft_cal', 'ft', 'useclass',
       'assignable_cal', 'assignable', 'transit', 'id', 'tollbooth',
       'bus_only', 'transit_access', 'managed', 'segment_id', 'tollseg',
       'distance', 'cntype', 'lanes_EA', 'lanes_AM', 'lanes_MD', 'lanes_PM',
       'lanes_EV'],
      dtype='object'). Returning 0.
Co

In [16]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.nodes_df.model_node_id.nunique())
print(model_net.shapes_df.shape)
print(model_net.shapes_df.id.nunique())

(2240283, 55)
2240283
(696500, 14)
696500
(1175025, 7)
1175025


In [17]:
model_net.links_df.rail_only.value_counts()

0    2239507
1        776
Name: rail_only, dtype: int64

In [18]:
model_net.links_df[(model_net.links_df.bus_only == 1)].cntype.unique()

array(['TANA'], dtype=object)

In [19]:
model_net.links_df.assignable.value_counts()

0    1282943
1     957340
Name: assignable, dtype: int64

In [20]:
model_net.links_df[model_net.links_df.cntype == "TAZ"].assignable.value_counts()

1    32986
Name: assignable, dtype: int64

In [21]:
model_net.links_df[model_net.links_df.cntype == "MAZ"].assignable.value_counts()

1    531534
Name: assignable, dtype: int64

In [22]:
model_net.links_df[model_net.links_df.A == 2601773].roadway

1313           motorway
4318          ml_access
803801    motorway_link
Name: roadway, dtype: object

In [23]:
model_net.links_df[model_net.links_df.model_link_id == 3152465].shstReferenceId

782651    e5742b085ba3dcb41da0c499cecc33bc
Name: shstReferenceId, dtype: object

In [24]:
# AC Transit node farezone
model_net.nodes_df[model_net.nodes_df.model_node_id == 1011480].farezone
model_net.nodes_df.farezone.max()

124

In [25]:
model_net.links_df.cntype.value_counts()

TANA     1006677
PED       631872
MAZ       531534
TAP        36438
TAZ        32986
CRAIL        776
Name: cntype, dtype: int64

In [26]:
model_net.nodes_mtc_df[[c for c in model_net.nodes_mtc_df.columns if c in parameters.output_variables]].columns

Index(['osm_node_id', 'county', 'drive_access', 'walk_access', 'bike_access',
       'N', 'rail_only', 'X', 'Y', 'geometry', 'farezone', 'tap_id'],
      dtype='object')

In [27]:
model_net.links_mtc_df[[c for c in model_net.links_mtc_df.columns if c in parameters.output_variables]].columns

Index(['bike_access', 'drive_access', 'name', 'shstGeometryId', 'walk_access',
       'county', 'model_link_id', 'A', 'B', 'rail_only', 'ft', 'assignable',
       'transit', 'tollbooth', 'bus_only', 'managed', 'segment_id', 'tollseg',
       'distance', 'cntype', 'lanes_EA', 'lanes_AM', 'lanes_MD', 'lanes_PM',
       'lanes_EV', 'useclass_EA', 'useclass_AM', 'useclass_MD', 'useclass_PM',
       'useclass_EV', 'geometry'],
      dtype='object')

In [28]:
model_net.links_mtc_df.county.value_counts()

Santa Clara      662698
Alameda          444167
Contra Costa     327165
San Mateo        203945
Sonoma           178969
Solano           157577
San Francisco    133492
Marin             85148
Napa              47068
External             54
Name: county, dtype: int64

In [29]:
model_net.nodes_mtc_df.county.value_counts()

Santa Clara      204654
Alameda          137350
Contra Costa     104856
San Mateo         61910
Sonoma            60031
Solano            50751
San Francisco     33407
Marin             28530
Napa              14990
External             21
Name: county, dtype: int64

In [30]:
model_net.links_mtc_df.shape

(2240283, 55)

In [31]:
model_net.links_mtc_df[(model_net.links_mtc_df.assignable == 1) & (model_net.links_mtc_df.ft==99)].roadway.value_counts()

service    29198
Name: roadway, dtype: int64

In [32]:
model_net.nodes_mtc_df.N.nunique()

696500

In [33]:
model_net.nodes_mtc_df.columns

Index(['index', 'osm_node_id', 'shst_node_id', 'county', 'drive_access',
       'walk_access', 'bike_access', 'N', 'rail_only', 'X', 'Y', 'geometry',
       'farezone', 'tap_id'],
      dtype='object')

In [34]:
# tollbooth
pd.crosstab(model_net.links_mtc_df.tollbooth, model_net.links_mtc_df.managed)

managed,0,1
tollbooth,,
0,2238768,1351
1,2,1
2,1,1
3,1,1
4,1,1
5,2,2
6,1,1
7,1,1
8,1,1


In [35]:
# tollseg
pd.crosstab(model_net.links_mtc_df.tollseg, model_net.links_mtc_df.managed)

managed,0,1
tollseg,,
0,2238776,1360
1,0,106
2,0,10
3,0,13
4,0,8
5,0,8
36,2,0


# Write to Disk

## create roadway files

In [36]:
model_net.links_mtc_df['name'] = model_net.links_mtc_df['name'].apply(lambda x: util.shorten_name(x))

In [37]:
model_net.links_mtc_df['name'].str.len().max()

85

In [38]:
model_net_filename = os.path.join(output_dir, 'model_net.pickle')
pickle.dump(model_net, open(model_net_filename, 'wb'))

In [115]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry", "cntype", "lanes_AM", "assignable", "useclass_AM", 'name', 'tollbooth', 'drive_access', 'bike_access', 'walk_access'],
    node_output_variables = ["model_node_id", "N", "geometry", "farezone", "tap_id"],
    data_to_csv = False,
    data_to_dbf = True,
)

2022-01-19 15:11:05, INFO: Writing Network as Shapefile
2022-01-19 15:11:05, INFO: Writing Network as Shapefile
2022-01-19 15:11:05, INFO: Renaming DBF Node Variables
2022-01-19 15:11:05, INFO: Renaming DBF Node Variables
2022-01-19 15:11:05, INFO: Renaming variables so that they are DBF-safe
2022-01-19 15:11:05, INFO: Renaming variables so that they are DBF-safe
2022-01-19 15:11:20, INFO: Renaming DBF Link Variables
2022-01-19 15:11:20, INFO: Renaming DBF Link Variables
2022-01-19 15:11:20, INFO: Renaming variables so that they are DBF-safe
2022-01-19 15:11:20, INFO: Renaming variables so that they are DBF-safe
2022-01-19 15:11:21, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_13\nodes.shp
2022-01-19 15:11:21, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_13\nodes.shp
2022-01-19 15:12:23, INFO: Writing Link Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\pro

z:\data\users\sijia\mtc\github\lasso\lasso\roadway.py:1565: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  links_dbf_df.to_file(output_link_shp)


## roadway Cube files

In [116]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

2022-01-19 15:18:28, INFO: Starting fixed width conversion
2022-01-19 15:18:28, INFO: Starting fixed width conversion
2022-01-19 15:35:02, INFO: Writing out link database
2022-01-19 15:35:02, INFO: Writing out link database
2022-01-19 15:35:21, INFO: Writing out link header and width ----
2022-01-19 15:35:21, INFO: Writing out link header and width ----
2022-01-19 15:35:21, INFO: Starting fixed width conversion
2022-01-19 15:35:21, INFO: Starting fixed width conversion
2022-01-19 15:37:04, INFO: Writing out node database
2022-01-19 15:37:04, INFO: Writing out node database
2022-01-19 15:37:07, INFO: Writing out node header and width
2022-01-19 15:37:07, INFO: Writing out node header and width


In [117]:
%%capture
os.chdir(output_dir)
!make_cube.bat

links.txt(1243000): 1;1;                                                     
links.txt(1244000): 1;1;                                                     
links.txt(1245000): 1;0;                                                     
links.txt(1246000): 1;0;                                                     
links.txt(1247000): 1;1;                                                     
links.txt(1248000): 1;0;                                                     
links.txt(1249000): 1;1;                                                     
links.txt(1250000): 1;1;                                                     
links.txt(1251000): 1;1;                                                     
links.txt(1252000): 1;0;                                                     
links.txt(1253000): 1;0;                                                     
links.txt(1254000): 1;0;                                                     
links.txt(1255000): 0;0;                                        

links.txt(1356000): 1;1;                                                     
links.txt(1357000): 0;0;                                                     
links.txt(1358000): 1;1;                                                     
links.txt(1359000): 1;0;                                                     
links.txt(1360000): 1;1;                                                     
links.txt(1361000): 1;1;                                                     
links.txt(1362000): 0;0;                                                     
links.txt(1363000): 1;0;                                                     
links.txt(1364000): 1;0;                                                     
links.txt(1365000): 1;1;                                                     
links.txt(1366000): 1;1;                                                     
links.txt(1367000): 1;0;                                                     
links.txt(1368000): 1;1;                                        

NODEMERGE: 2152084

NODEMERGE: 2153084

NODEMERGE: 2154084

NODEMERGE: 2155084

NODEMERGE: 2156084

NODEMERGE: 2157084

NODEMERGE: 2158084

NODEMERGE: 2159084

NODEMERGE: 2160084

NODEMERGE: 2161084

NODEMERGE: 2162084

NODEMERGE: 2163084

NODEMERGE: 2164084

NODEMERGE: 2165084

NODEMERGE: 2166084

NODEMERGE: 2167084

NODEMERGE: 2168084

NODEMERGE: 2169084

NODEMERGE: 2170084

NODEMERGE: 2171084

NODEMERGE: 2172084

NODEMERGE: 2173084

NODEMERGE: 2174084

NODEMERGE: 2175084

NODEMERGE: 2176084

NODEMERGE: 2177084

NODEMERGE: 2178084

NODEMERGE: 2179084

NODEMERGE: 2180084

NODEMERGE: 2181084

NODEMERGE: 2182084

NODEMERGE: 2183084

NODEMERGE: 2184084

NODEMERGE: 2185084

NODEMERGE: 2186084

NODEMERGE: 2187084

NODEMERGE: 2188084

NODEMERGE: 2189084

NODEMERGE: 2190084

NODEMERGE: 2191084

NODEMERGE: 2192084

NODEMERGE: 2500135

NODEMERGE: 2501135

NODEMERGE: 2502135

NODEMERGE: 2503135

NODEMERGE: 2504135

NODEMERGE: 2505135

NODEMERGE: 2506135

NODEMERGE: 2507135

NODEMERGE: 2508135




LINKMERGE: 2005251-2169566     

LINKMERGE: 2005542-2070058     

LINKMERGE: 2005841-211693     

LINKMERGE: 2006137-2024130     

LINKMERGE: 2006432-2052434     

LINKMERGE: 2006722-2078180     

LINKMERGE: 2007018-2152561     

LINKMERGE: 2007309-2064959     

LINKMERGE: 2007601-229216     

LINKMERGE: 2007901-2040726     

LINKMERGE: 2008198-2083896     

LINKMERGE: 2008490-230609     

LINKMERGE: 2008788-2086210     

LINKMERGE: 2009075-2044630     

LINKMERGE: 2009370-2004364     

LINKMERGE: 2009660-2145739     

LINKMERGE: 2009953-2032490     

LINKMERGE: 2010257-2154988     

LINKMERGE: 2010558-2158078     

LINKMERGE: 2010860-2152319     

LINKMERGE: 2011162-2077376     

LINKMERGE: 2011459-2057107     

LINKMERGE: 2011752-2130356     

LINKMERGE: 2012048-231490     

LINKMERGE: 2012341-2162113     

LINKMERGE: 2012629-2126931     

LINKMERGE: 2012920-2016060     

LINKMERGE: 2013222-2156283     

LINKMERGE: 2013520-2024334     

LINKMERGE: 2013816-2168510     

LINKMERGE: 20


LINKMERGE: 3080248-3071182     

LINKMERGE: 3080605-3032743     

LINKMERGE: 3080968-3036804     

LINKMERGE: 3081596-3046451     

LINKMERGE: 3082269-3008897     

LINKMERGE: 3082920-410465     

LINKMERGE: 3083579-415330     

LINKMERGE: 3084259-3013700     

LINKMERGE: 3084930-425569     

LINKMERGE: 3085627-412408     

LINKMERGE: 3086309-3035534     

LINKMERGE: 3086993-3039568     

LINKMERGE: 3087679-3018661     

LINKMERGE: 3088375-410986     

LINKMERGE: 3089090-3023427     

LINKMERGE: 3089776-3054524     

LINKMERGE: 3090473-3057785     

LINKMERGE: 3091178-3045193     

LINKMERGE: 3091865-3039917     

LINKMERGE: 3092573-3028998     

LINKMERGE: 3093293-3047477     

LINKMERGE: 3094013-3072055     

LINKMERGE: 3094735-3072952     

LINKMERGE: 3095443-3021665     

LINKMERGE: 3096149-3058472     

LINKMERGE: 3096884-417289     

LINKMERGE: 3500113-3534000     

LINKMERGE: 3500410-3509105     

LINKMERGE: 3500723-3537335     

LINKMERGE: 3501025-3504555     

LINKMERGE: 3501

## Create transit files

In [41]:
transfer_fare_df = pd.read_csv(os.path.join(output_dir, "transfer.csv"))

In [43]:
mtc.write_cube_fare_files(
    roadway_network=model_net,
    transit_network=step5_scenario.transit_net,
    parameters=parameters,
    outpath = os.path.join(output_dir),
    transfer_fare_df = transfer_fare_df
)

In [44]:
standard_transit_net = StandardTransit.fromTransitNetwork(step5_scenario.transit_net, parameters = parameters)

Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [45]:
standard_transit_net_filename = os.path.join(output_dir, 'model_transit_net.pickle')
pickle.dump(standard_transit_net, open(standard_transit_net_filename, 'wb'))

## As transit line file (for CUBE)

In [125]:
for agency in standard_transit_net.feed.routes.agency_raw_name.unique():
    sub_transit_net = copy.deepcopy(standard_transit_net)
    sub_transit_net.feed.trips = sub_transit_net.feed.trips[sub_transit_net.feed.trips.agency_raw_name == agency]
    mtc.write_as_cube_lin(sub_transit_net, parameters, outpath = os.path.join(output_dir, agency + "_transit.lin"))

2022-01-20 12:31:55, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2022-01-20 12:31:55, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2022-01-20 12:31:56, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2022-01-20 12:31:56, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2022-01-20 12:33:35, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2022-01-20 12:33:35, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2022-01-20 12:33:43, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2022-01-20 12:33:43, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2022-01-20 12:33:45, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2022-01-20 12:33:45, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2022-01-20 12:33:49, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2022-01-20 12:33:49, INFO: Converting GTFS Standard Pr

In [126]:
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2022-01-20 12:39:36, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2022-01-20 12:39:36, INFO: Converting GTFS Standard Properties to MTC's Cube Standard


# write EMME
## this part writes out EMME networks, which require EMME license

In [46]:
from lasso import emme

In [9]:
# read pickle
model_net_pickle_file_name = os.path.join(output_dir, 'model_net.pickle')
model_net = pickle.load(open(model_net_pickle_file_name, 'rb'))

tranit_net_pickle_file_name = os.path.join(output_dir, 'model_transit_net.pickle')
standard_transit_net = pickle.load(open(tranit_net_pickle_file_name, 'rb'))

In [ ]:
emme.create_emme_network(
    roadway_network=model_net,
    transit_network=standard_transit_net,
    path=os.path.join(output_dir, 'emme_test071223'),
    write_drive_network = False,
    write_maz_active_modes_network = False,
    write_tap_transit_network = False,
    write_taz_transit_network = True,
    parameters=parameters,
    polygon_file_to_split_active_modes_network=os.path.join(lasso_dir, 'mtc_data', 'emme', 'subregion_boundary_for_active_modes.shp'),
    polygon_variable_to_split_active_modes_network = 'subregion'
)

Creating shapes for backward directions on two-way links
Make sure the CRS of input network is correct
Converting geometry into wkt geometry


In [126]:
model_net.links_mtc_df.transit.value_counts()

0    1636980
      602471
1        832
Name: transit, dtype: int64

In [141]:
model_net.links_mtc_df[(model_net.links_mtc_df.A == 318969) & (model_net.links_mtc_df.B == 2551228)]

,index,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,ML_lanes_AM,ML_lanes_MD,ML_lanes_PM,ML_lanes_EV,useclass_EA,useclass_AM,useclass_MD,useclass_PM,useclass_EV,geometry
2077812,2073309,0,1,,1,,,,,maz,...,0,0,0,0,0,0,0,0,0,"LINESTRING (4877132.307 3668469.261, 4877178.0..."


In [129]:
model_net.links_mtc_df[model_net.links_mtc_df.model_link_id == 2522827][['A', 'B', 'A_consecutive', 'B_consecutive']]

,A,B,A_consecutive,B_consecutive
1661625,200533,2094466,11371,229382


## As pickle

In [98]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_12.pickle')
pickle.dump(step5_scenario, open(working_scenario_filename, 'wb'))

## As standard network

In [99]:
step5_scenario.road_net.write(path = output_dir, filename = "v12")
step5_scenario.transit_net.write(path = output_dir, filename = "v12")

2021-09-30 16:30:55, INFO: Writing transit to directory: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12
2021-09-30 16:30:55, INFO: Writing transit to directory: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12


In [100]:
# save out link feather
# to_feather() does not like mixed type

links_feather = step5_scenario.road_net.links_df.copy()
links_feather.drop("geometry", axis = 1, inplace = True)

for c in links_feather.columns:
    if c in parameters.int_col:
        links_feather[c] = links_feather[c].astype(int)
    else:
        links_feather[c] = links_feather[c].astype(str)
        
links_feather.to_feather(path = os.path.join(output_dir, "v12_links.feather"))

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\ipykernel_launcher.py:13: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  del sys.path[0]


In [101]:
stop_node_id = step5_scenario.transit_net.feed.stops.model_node_id.tolist()

In [102]:
step5_scenario.road_net.nodes_df[step5_scenario.road_net.nodes_df.model_node_id.isin(stop_node_id)].walk_access.value_counts()

1    18215
Name: walk_access, dtype: int64

In [103]:
model_net.links_df[model_net.links_df.model_link_id == 3033292].assignable

175454    1
Name: assignable, dtype: int32

In [104]:
model_net.nodes_df.columns

Index(['index', 'osm_node_id', 'shst_node_id', 'county', 'drive_access',
       'walk_access', 'bike_access', 'model_node_id', 'rail_only', 'X', 'Y',
       'geometry', 'farezone', 'tap_id'],
      dtype='object')

In [105]:
step5_scenario.transit_net.feed.stops.model_node_id.nunique()

18215